# Кодирование методом Фибоначчи


Используем ряд чисел Фибонначи как основания псевдо-ЧС.
Естественным терминатором закодированной последовательности является **"11"**

In [3]:
class BitstreamWriter:
    def __init__(self):
        self.nbits  = 0
        self.curbyte = 0
        self.vbytes = []

    def add(self, x):
        self.curbyte |= x << (8-1 - (self.nbits % 8))
        self.nbits += 1

        if self.nbits % 8 == 0:
            self.vbytes.append(chr(self.curbyte))
            self.curbyte = 0

    def getbytes(self):
        if self.nbits & 7 == 0:
            return "".join(self.vbytes)

        return "".join(self.vbytes) + chr(self.curbyte)


class BitstreamReader:
    def __init__(self, blob):
        self.blob = blob
        self.pos  = 0

    def get(self):
        ibyte = self.pos / 8
        ibit  = self.pos & 7

        self.pos += 1
        return (ord(self.blob[ibyte]) & (1 << (7 - ibit))) >> (7 - ibit)

    def finished(self):
        return self.pos >= len(self.blob) * 8


In [117]:
def gen_fib(upper_bound):
    f1 = 1
    f2 = 2
    l = []
    l.append(f1)
    l.append(f2)
    now_num = f1 + f2;
    while (now_num <= upper_bound):
        l.append(now_num)
        now_num =l[len(l) - 1] + l[len(l) - 2] 
    return l

def fib_count(count):
    f1 = 1
    f2 = 2
    l = []
    if (count == 1):
        l.append(f1)
        return l
    elif (count == 2):
        l.append(f2)
        return l
    l.append(f1)
    l.append(f2)
    while (len(l) < count):
        l.append(l[len(l) - 1] + l[len(l) - 2])
    return l
    

def compress_fblist(l):
    bs = BitstreamWriter()
    for num in l:
        fib = gen_fib(num)
        fib = list(reversed(fib))
        pos_arr = 0
        n1 = num
        now_seq = []

        while (n1 > 0 and pos_arr < len(fib)):
            if (n1 >= fib[pos_arr]):
                n1 -= fib[pos_arr]
                now_seq.append(1)
            else:
                now_seq.append(0)
            pos_arr += 1
        while (pos_arr < len(fib)):
            now_seq.append(0)
            pos_arr += 1
        now_seq = list(reversed(now_seq))
        now_seq.append(1)
        #print now_seq
        #print now_seq
        
        for k in now_seq:
            bs.add(k)
        
    return bs.getbytes()


def decompress_fblist(s):
    bs = BitstreamReader(s)
    result = []
    fl = True
    while (fl):
        now_arr = []
        now = bs.get()
        prev = -1
        fl = bs.finished()
        while (fl):
            now_arr.append(now)
            prev = now
            now = bs.get()
            if (now == prev and now == 1):
                break
            fl = not bs.finished()
            print now, prev
        
        fib = fib_count(len(now_arr))
        n = 0
        print now_arr
        for p in range(len(fib)):
            n += fib[p] * now_arr[p]
        result.append(n)
        if (not fl):
            break

    return result

In [118]:
c = compress_fblist([10, 22, 43])
nums = decompress_fblist(c)
print nums

[]


IndexError: list index out of range

In [76]:
import unittest
import random

class TestFibbonacciCompression(unittest.TestCase):
    def test_simple_compression(self):
        cb = compress_fblist([4, 48, 115, 190])
        nums = decompress_fblist(cb)
        self.assertListEqual(nums, [4, 48, 115, 190])


    def test_one_element(self):
        cb = compress_fblist([40])
        nums = decompress_fblist(cb)
        self.assertListEqual(nums, [40])

    def test_encode_zero(self):
        cb = compress_fblist([0])
        decoded_zero = decompress_fblist(cb)
        self.assertListEqual(decoded_zero, [0])

    def test_random_elements(self):
        n = random.randint(1000, 2000)
        arr = []
        for i in xrange(n):
            delta = random.randint(1, 100)
            arr.append(delta if i == 0 else arr[i-1]+delta)
        cb = compress_fblist(arr)
        decoded = decompress_fblist(cb)
        self.assertListEqual(decoded, arr)


    def test_zeroID(self):
        # there is a problem with encoding of ZERO in fibbonacci code.
        # so encoding should do something with it. And decode result transparently
        cb = compress_fblist([0, 5, 8])
        nums = decompress_fblist(cb)
        
    def test_fail_if_unsorted(self):
        with self.assertRaises(IndexError):
            compress_fblist([8, 1])


    def test_encoding(self):
        cb = compress_fblist([0, 1, 2, 3])
        self.assertEqual(len(cb), 1)
        self.assertEqual(cb[0], chr(0b11111111))

        cb = compress_fblist([0, 1, 2, 3, 4])
        self.assertEqual(len(cb), 2)
        self.assertEqual(cb[0], chr(0b11111111))
        self.assertEqual(cb[1], chr(0b11000000))

suite = unittest.TestLoader().loadTestsFromTestCase(TestFibbonacciCompression)
unittest.TextTestRunner().run(suite)

FFFFFF.
FAIL: test_encode_zero (__main__.TestFibbonacciCompression)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-76-c82a989b143e>", line 19, in test_encode_zero
    self.assertListEqual(decoded_zero, [0])
AssertionError: Lists differ: [3] != [0]

First differing element 0:
3
0

- [3]
+ [0]

FAIL: test_encoding (__main__.TestFibbonacciCompression)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-76-c82a989b143e>", line 45, in test_encoding
    self.assertEqual(len(cb), 1)
AssertionError: 2 != 1

FAIL: test_fail_if_unsorted (__main__.TestFibbonacciCompression)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-76-c82a989b143e>", line 40, in test_fail_if_unsorted
    compress_fblist([8, 1])
AssertionError: IndexError not raised

FAIL: test_one_element

<unittest.runner.TextTestResult run=7 errors=0 failures=6>